In [1]:
from google.colab import drive
drive.mount("/content/drive/")
drive_path="/content/drive/Mydrive"

Mounted at /content/drive/


In [2]:
!git clone https://github.com/PKU-YuanGroup/LanguageBind
!cd LanguageBind
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
#!pip install -r requirements.txt
!pip install einops
!pip install peft
!pip install transformers==4.31.0
!pip install decord
!pip install pytorchvideo
!pip install urlextract

Cloning into 'LanguageBind'...
remote: Enumerating objects: 1347, done.
remote: Counting objects: 100% (371/371), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 1347 (delta 264), reused 242 (delta 173), pack-reused 976
Receiving objects: 100% (1347/1347), 18.98 MiB | 31.45 MiB/s, done.
Resolving deltas: 100% (720/720), done.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 923.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 110.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu118
    Uninstalling torchvision-0.16.0+cu118:
      Successfully un

In [3]:
%cd /content/LanguageBind

/content/LanguageBind


In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# from languagebind import LanguageBindVideo, LanguageBindVideoTokenizer, LanguageBindVideoProcessor
# from languagebind import LanguageBindAudio, LanguageBindAudioTokenizer, LanguageBindAudioProcessor
# from languagebind import LanguageBindImage,  LanguageBindImageTokenizer,  LanguageBindImageProcessor
from urlextract import URLExtract
# from google.colab import output
import urllib.request
import moviepy.editor
from moviepy.editor import VideoFileClip

In [8]:
pretrained_ckpt = 'LanguageBind/LanguageBind_Video_FT'  # also 'LanguageBind/LanguageBind_Video'
model_v = LanguageBindVideo.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir')
tokenizer_v = LanguageBindVideoTokenizer.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir')
video_process = LanguageBindVideoProcessor(model_v.config, tokenizer_v)

In [14]:
pretrained_ckpt = 'LanguageBind/LanguageBind_Audio'  # also 'LanguageBind/LanguageBind_Audio'
model_a = LanguageBindAudio.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir')
tokenizer_a = LanguageBindAudioTokenizer.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir')
audio_process = LanguageBindAudioProcessor(model_a.config, tokenizer_a)

Some weights of LanguageBindAudio were not initialized from the model checkpoint at LanguageBind/LanguageBind_Audio and are newly initialized: ['vision_model.encoder.base_model.model.layers.22.self_attn.k_proj.base_layer.bias', 'vision_model.encoder.base_model.model.layers.16.self_attn.out_proj.base_layer.bias', 'vision_model.encoder.base_model.model.layers.1.self_attn.q_proj.base_layer.bias', 'vision_model.encoder.base_model.model.layers.8.self_attn.k_proj.base_layer.weight', 'vision_model.encoder.base_model.model.layers.3.self_attn.v_proj.base_layer.weight', 'vision_model.encoder.base_model.model.layers.5.self_attn.v_proj.base_layer.bias', 'vision_model.encoder.base_model.model.layers.7.self_attn.k_proj.base_layer.weight', 'vision_model.encoder.base_model.model.layers.13.self_attn.k_proj.base_layer.bias', 'vision_model.encoder.base_model.model.layers.3.self_attn.q_proj.base_layer.weight', 'vision_model.encoder.base_model.model.layers.0.self_attn.k_proj.base_layer.weight', 'vision_mod

In [15]:
pretrained_ckpt = 'LanguageBind/LanguageBind_Image'
model_i = LanguageBindImage.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir')
tokenizer_i = LanguageBindImageTokenizer.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir')
image_process = LanguageBindImageProcessor(model_i.config, tokenizer_i)

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/814 [00:00<?, ?B/s]

In [21]:
train_df=pd.read_csv("/content/drive/MyDrive/Adobe/train.csv")

In [22]:
train_df.head()

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,6,2020-11-15 16:01:08,525,An 85-year-old primary school in Shanghai has ...,cnni,[Video(thumbnailUrl='https://pbs.twimg.com/amp...,cnn


In [60]:
extractor = URLExtract()
example_text = train_df["media"][1]

for url in extractor.gen_urls(example_text):
    print(url) # prints: ['janlipovsky.cz']

https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=small
https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=large


In [61]:
list(extractor.gen_urls(example_text))

['https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=small',
 'https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=large']

In [30]:
output.clear()

In [33]:
%cd /content

/content


In [46]:
url

'https://video.twimg.com/amplify_video/1322382278794858496/vid/480x270/5Fx4A4yzGMvLpTUH.mp4?tag=13'

In [50]:
urllib.request.urlretrieve(url,"vid.mp4")
data = video_process(["/content/vid.mp4"], ["text"], return_tensors='pt')

In [5]:
def encode_video(url):
  if('/vid/' not in url):
    print("not a video")
    return
  urllib.request.urlretrieve(url,"vid.mp4")
  print("Got video")
  data = video_process(["/content/vid.mp4"], ["text"], return_tensors='pt')
  print("Processed video")
  #data = {key: val.to(device) for key, val in data.items()}  # Move data to GPU
  model_v.eval()

  with torch.no_grad():
      out = model_v(**data)
      return out.image_embeds  # Return image embeddings
  print("Got embeddings")

In [59]:
vid_embeds = encode_video('https://video.twimg.com/amplify_video/1322382278794858496/vid/480x270/5Fx4A4yzGMvLpTUH.mp4?tag=13')

Got video
Processed video


In [63]:
def encode_image(url):
  if('/vid/' in url):
    print("not an image")
    return
  urllib.request.urlretrieve(url,"img.jpg")
  print("Got image")
  data = image_process(["/content/img.jpg"], ["text"], return_tensors='pt')
  print("Processed image")
  #data = {key: val.to(device) for key, val in data.items()}  # Move data to GPU
  model_i.eval()

  with torch.no_grad():
      out = model_i(**data)
      return out.image_embeds  # Return image embeddings
  print("Got embeddings")

In [64]:
img_embeds = encode_image('https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?format=jpg&name=small')

Got image
Processed image


In [6]:
def encode_audio(url):
  if('/vid/' not in url):
    print("not a video")
    return
  urllib.request.urlretrieve(url,"vid.mp4")
  print("Got video")

  video = moviepy.editor.VideoFileClip("vid.mp4")
  audio = video.audio
  audio.write_audiofile("audio.wav")
  print("Got audio")

  data = audio_process(["/content/audio.wav"], ["text"], return_tensors='pt')
  print("Processed audio")
  #data = {key: val.to(device) for key, val in data.items()}  # Move data to GPU
  model_a.eval()

  with torch.no_grad():
      out = model_a(**data)
      return out.image_embeds  # Return image embeddings
  print("Got embeddings")

In [8]:
aud_embeds = encode_audio('https://video.twimg.com/amplify_video/1019341984543268866/vid/640x360/ewrbY6gdakX6772y.mp4?tag=3')

Got video
MoviePy - Writing audio in audio.wav


MoviePy - Done.
Got audio


NameError: name 'audio_process' is not defined

Now we have the code for getting embeddings of image, video and audio given the url of the datapoint.

In [76]:
aud_embeds.shape,img_embeds.shape,vid_embeds.shape

(torch.Size([1, 768]), torch.Size([1, 768]), torch.Size([1, 768]))

For the NeXT GPT architecture, we need another linear layer to input into the LLM.